# Part 3: リモートデータを操作するための高度ツール

前回のセクションではFederated Learningを使ってトイモデル（初歩的なモデル）の学習を行いました。`.send()`や`.get()`を使う事でモデルをデータのある場所へ送ってパラメータをアップデートしたり、学習済みモデルを受け取ったりできることを確認しました。しかし、前回の最後になって、プライバシーを守るためにはそれだけでは不十分だということに気がつきました。もう少し正確に言うと、`.get()`する前に勾配ベクトルを平均する必要があるということです。`.get()`実行前にワーカー全員の勾配ベクトルの平均を取れれば、もう個別のワーカーの勾配ベクトルを知る事はできません。（結果としてプライバシーの保護レベルを高めることができます。）

しかし、このテクニックを実行するためには、次の事ができなくてはなりません。

- ポインタtensorのAPIを使って、あるワーカーから別のワーカーへtensorを直接送れること

本セクションでは、上記に加え、今後のチュートリアルで使用するいくつかの高度なオペレーションについても学びます。

Authors:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

In [ ]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

# Section 3.1 - ポインタのポインタ

ここまでに学習した通り、PointerTensorは通常のtensorと同じような扱いが可能です。実際、PointerTensorとtensorは実に良く似ているので、そのままPointerTensorのポインタを作る事もできます。実際に確認してみましょう。

In [ ]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [ ]:
# 通常のtensorです。ローカルに存在します。
x = torch.tensor([1,2,3,4])
x

In [ ]:
# tensorをBobへ送ります。戻り値としてPointerTensorを受け取ります。
x_ptr = x.send(bob)

# Bobが持っているtensorへのポインタです。
x_ptr

In [ ]:
# ではPointerTensorをAliceへ送ってみましょう。
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

### 何がどうなった？

直前の例では、私たちは`x`という名前でtensorを作成し、Bobへ送りました。私たちは`x_ptr`という名前でポインタを保持しています。 

さらに、`x_ptr.send(alice)`を実行し、**ポインタ**をAliceへ送りました。

注記: ポインタを第三者へ送ってもデータそのものは移動されません。データへのポインタだけが送られています。

In [ ]:
# 上記の通り、Bobがまだ実際のデータを保持しています。（実データは常にtensor型で保持されます）
bob._objects

In [ ]:
# 一方でAliceはx_ptrを保持しています。（x_ptrがBob保有のtensorを参照していることを確認してください）
alice._objects

In [ ]:
# そして、私たちは.get()を使う事でAliceからx_ptrを受け取る（返してもらう？）ことができます。
x_ptr = pointer_to_x_ptr.get()
x_ptr

In [ ]:
# そして、x_ptrに対して.get()を実行することでBobからxを受け取る（返してもらう）ことができます。

x = x_ptr.get()
x

### ポインタのポインタを使っての演算

通常のポインタと同様、私たちはポインタのポインタに対してもPyTorchの演算処理を実行することが可能です。

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
p2p2x = torch.tensor([1,2,3,4,5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
y.get().get()

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
p2p2x.get().get()

In [ ]:
bob._objects

In [ ]:
alice._objects

# Section 3.2 - ポインタにおけるチェーンオペレーション

ここまでの例では、`.send()`や`.get()`を使う事でローカルマシンからワーカーへtensorやポインタを送信したりワーカーからtensorやポインタを受信したりしてきました。しかし、ポインタのチェーンを扱うようになってくると、あるワーカーから別のワーカーへ直接tensorを送って欲しいことがあります。そうなってくると、そのために特別にデザインされた特別な関数が欲しくなりますよね。

そんなオペレーションの一つは

- `my_pointer2pointer.move(another_worker)`です。

In [ ]:
# xはBobのマシンにあるtensorのポインタです。
x = torch.tensor([1,2,3,4,5]).send(bob)

In [ ]:
x

In [ ]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

In [ ]:
x = x.move(alice)

In [ ]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

エクセレント！ これにより、信頼できるアグリゲーター（集計者）による**gradient averaging**（勾配ベクトルの平均化）の準備が整いました。

# おめでとうございます！コミュニティに入ろう！

本チュートリアルを完了しました。おめでとうございます！もし、このチュートリアルを気に入って、プライバシーに配慮した非中央集権的なAI技術や付随する（データやモデルの）サプライチェーンにご興味があって、プロジェクトに参加したいと思われるなら、以下の方法で可能です。

### PySyftのGitHubレポジトリにスターをつける

一番簡単に貢献できる方法はこのGitHubのレポジトリにスターを付けていただくことです。スターが増えると露出が増え、より多くのデベロッパーにこのクールな技術の事を知って貰えます。

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Slackに入る

最新の開発状況のトラッキングする一番良い方法はSlackに入ることです。
下記フォームから入る事ができます。
[http://slack.openmined.org](http://slack.openmined.org)

### コードプロジェクトに参加する

コミュニティに貢献する一番良い方法はソースコードのコントリビューターになることです。PySyftのGitHubへアクセスしてIssueのページを開き、"Projects"で検索してみてください。参加し得るプロジェクトの状況を把握することができます。また、"good first issue"とマークされているIssueを探す事でミニプロジェクトを探すこともできます。

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### 寄付

もし、ソースコードで貢献できるほどの時間は取れないけど、是非何かサポートしたいという場合は、寄付をしていただくことも可能です。寄附金の全ては、ハッカソンやミートアップの開催といった、コミュニティ運営経費として利用されます。

[OpenMined's Open Collective Page](https://opencollective.com/openmined)